In [4]:
import numpy as np
from torch import nn, optim
import torch
import random
import time

🤔思考：如何加载数据？

In [5]:
with open("labels.txt", "r") as f:
    raw = f.readlines()

tags = []
data = []
for l in raw:
    tags.append(int(l[0]))#每行的第一个字符是标签
    d = l[1:-1]#去掉标签和换行符
    d = map(float, tuple(d)) #将字符串转换为tuple，数字转换为float，方便后续转为tensor
    #tuple相对于list更省内存，因为tuple是不可变的，对象所含method更少
    data.append(tuple(d))

#将标签和数据转为tensor，方便后续切分训练集和测试集
data = torch.tensor(data)
tags = torch.tensor(tags)

#划分训练集和测试集
train_test_ratio = 0.8
train_size = int(train_test_ratio * len(data))
test_size = len(data) - train_size
data_train = data[:train_size]
data_test  = data[train_size:]
tags_train = tags[:train_size]
tags_test  = tags[train_size:]

In [6]:
# 直接套用d2l网站上的代码，没有改动
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


batch_size = 10

w = torch.normal(0, 0.01, size=(len(data[0]), 1), requires_grad=True, dtype=torch.float32) #对每个像素都有一个权重
b = torch.zeros(1, requires_grad=True)

👇方案3：Multi-Layer Perceptron (MLP)

In [7]:
net = nn.Sequential(nn.Linear(384, 128),
                    nn.ReLU(),
                    nn.Linear(128, 10))
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)  # 初始化权重
batch_size, lr, num_epochs = 32, 0.1, 16  # 设置超参数
loss = nn.CrossEntropyLoss(reduction='none') # 定义损失函数
trainer = torch.optim.SGD(net.parameters(), lr=lr) # 定义优化器

In [8]:
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, data_train, tags_train):
        y_hat = net(X)
        l = loss(y_hat, y)
        l = l.mean()
        trainer.zero_grad()
        l.backward()
        trainer.step()
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter(batch_size, data_test, tags_test):
            y_hat = net(X)
            acc_sum += (y_hat.argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
    # n = test_size
    test_acc = acc_sum / n
    print('epoch %d, test acc %.3f' % (epoch + 1, test_acc))

epoch 1, test acc 0.140
epoch 2, test acc 0.430
epoch 3, test acc 0.837
epoch 4, test acc 0.965
epoch 5, test acc 0.965
epoch 6, test acc 0.965
epoch 7, test acc 0.965
epoch 8, test acc 0.965
epoch 9, test acc 0.965
epoch 10, test acc 0.965
epoch 11, test acc 0.965
epoch 12, test acc 0.965
epoch 13, test acc 0.965
epoch 14, test acc 0.965
epoch 15, test acc 0.965
epoch 16, test acc 0.965
